# Прогнозирование изменения ставки ЦБ (Baseline-решение)

### Описание проекта:

Baseline-решение задачи классификации направления изменения будущей ключевой ставки ЦБ РФ по семантике текущего пресс-релиза, величины инфляции, курса доллара, а также текущей ставки ЦБ.

### Цели проекта:

Цель данной работы построение бейзлайна и его улучшение с помощью простейших моделей, для предсказания направления изменения ставки рефинансирования ЦБ. Это наша отправная точка для сравнения и улучшения. 

### План работы:

1. Минимальный бейзлайн.
2. Статистические методы :
    - Мешок слов плюс линейная модель
    - TF-IDF плюс линейная модель
    - N-граммы плюс Naive Bayes
3. Создание эмбеддингов:
    - Word2Vec плюс линейная модель
    - GloVe плюс линейная модель
4. Выводы

### Описание данных:

- `date` - дата публикации пресс-релиза
- `title` - заголовок пресс-релиза
- `release` - текст пресс-релиза
- `inflation` - годовая инфляция
- `rate` - величина ставки рефинансирования ЦБ, объявленная на следущем заседании
- `usd` - курс доллара
- `usd_cur_change_relative` - относительное изменение курса доллара, по сравнению с предыдущим заседанием
- `target_categorial` - категориальная метка направления изменения ставки рефинансирования ЦБ, 1 - повышение, -1 - понижение, 0 - без изменений.
- `target_absolute` - абсолютное изменение ставки рефинансирования ЦБ (следующее значение минус текущее)
- `target_relative` - относительное изменение ставки рефинансирования ЦБ (следующее значение делится на текущее)

In [62]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold, cross_validate, KFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from string import punctuation
from pymystem3 import Mystem
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

RANDOM_STATE = 41825352
DATASET_URL = '../data/cbr-press-releases.csv'

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /home/father/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/father/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/father/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

### Загрузка данных

Загрузим датасет пресс-релизов, полученных с сайта ЦБ РФ:

In [63]:
df = pd.read_csv(DATASET_URL, parse_dates=['date'])

Дату сделаем индексом, ссылку на пресс-релиз выбросим, эта переменная не поможет в предсказании таргета.

In [64]:
df.set_index('date', inplace=True)
df.drop('link', axis=1, inplace=True)


Для последнего релиза неизвестна целевая переменная (направление изменения ключевой ставки), поэтому исключим его из датасета и сохраним отдельно:

In [66]:
df.sort_values('date', inplace=True)
cur_pr = df.tail(1)
df = df[:-1]
df

,title,release,inflation,rate,usd,usd_cur_change_relative,target_categorial,target_absolute,target_relative
date,,,,,,,,,
2013-09-13,О процентных ставках по операциям Банка России,Департамент внешних и общественных связей Банк...,6.51,5.5,32.6731,0.994294,0.0,0.0,1.000000
2013-10-14,О ключевой ставке Банка России,"Пресс-служба Банка России сообщает, что Совет ...",6.14,5.5,32.2663,0.987549,0.0,0.0,1.000000
2013-11-08,О ключевой ставке Банка России,Совет директоров Банка России 8 ноября 2013 го...,6.27,5.5,32.3803,1.003533,0.0,0.0,1.000000
2013-12-13,О ключевой ставке Банка России,Совет директоров Банка России 13 декабря 2013 ...,6.50,5.5,32.7518,1.011473,0.0,0.0,1.000000
2014-02-14,О ключевой ставке Банка России,Совет директоров Банка России 14 февраля 2014 ...,6.07,7.0,34.8611,1.064403,1.0,1.5,1.272727
...,...,...,...,...,...,...,...,...,...
2024-02-16,Банк России принял решение сохранить ключевую ...,Совет директоров Банка России 16 февраля 2024 ...,7.44,16.0,91.8237,1.023971,0.0,0.0,1.000000
2024-03-22,Банк России принял решение сохранить ключевую ...,Совет директоров Банка России 22 марта 2024 го...,7.69,16.0,91.9499,1.001374,0.0,0.0,1.000000
2024-04-26,Банк России принял решение сохранить ключевую ...,Совет директоров Банка России 26 апреля 2024 г...,7.72,16.0,92.1314,1.001974,0.0,0.0,1.000000


### Предобработка текстовых переменных

In [6]:
import re


mystem = Mystem()

def preprocessor(text):
    regex = re.compile('[^а-я А-ЯЁё]')
    text = regex.sub(' ', text)
    return ' '.join(mystem.lemmatize(text))

vectorizer = CountVectorizer(
    lowercase=True,
    stop_words=list(stopwords.words('russian')),
    tokenizer=word_tokenize,
    preprocessor=preprocessor,
)

vectorizer.fit(df.release)

bow = vectorizer.transform(df.release)

/home/father/Desktop/cbr-press-release-classifier/.venv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/father/Desktop/cbr-press-release-classifier/.venv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['весь', 'свой', 'это'] not in stop_words.
  warnings.warn(


В качестве целевой переменной используется столбец `target_categorial`.

In [7]:
y = df.target_categorial

In [46]:
def calc_metrics(bow, penalty, solver='lbfgs'):
    y_preds = []
    for threshold in range(30, 96):
        lr = LogisticRegression(penalty=penalty, solver=solver, C=1, max_iter=10000)
        X_train = bow[:threshold]
        X_test = bow[threshold:]
        y_train = y[:threshold]
        y_test = y[threshold:]

        lr.fit(X_train, y_train)
        y_pred = lr.predict(X_test[0].reshape(1, -1))
        y_preds.append(y_pred)
        
    return accuracy_score(y[30:], y_preds), lr
    
acc, lr = calc_metrics(bow, 'l1', 'liblinear')
acc

0.6363636363636364

In [47]:
calc_metrics(bow[:, np.any(lr.coef_ != 0, axis=0)], 'l2', 'newton-cg')

(0.6818181818181818,
 LogisticRegression(C=1, max_iter=10000, solver='newton-cg'))

In [48]:
vectorizer.get_feature_names_out()[np.any(lr.coef_ != 0, axis=0)]

array(['активность', 'апрель', 'банк', 'банковский', 'близкий', 'вблизи',
       'влияние', 'внешний', 'внутренний', 'возможность',
       'восстановление', 'временный', 'время', 'год', 'декабрь',
       'денежный', 'депозитный', 'допускать', 'замедление', 'инфляция',
       'итог', 'июль', 'июнь', 'квартал', 'конец', 'который', 'мера',
       'месяц', 'мочь', 'наблюдаться', 'ндс', 'неопределенность',
       'несколько', 'низкий', 'ноябрь', 'ограничение', 'ожидание',
       'оказывать', 'октябрь', 'операция', 'оставаться', 'оценка', 'п',
       'первый', 'повышение', 'поддерживаться', 'пока', 'показатель',
       'потребительский', 'предприятие', 'принимать', 'прирост',
       'продолжать', 'происходить', 'процентный', 'расти', 'расход',
       'риск', 'российский', 'россия', 'рост', 'рубль', 'сектор',
       'сентябрь', 'смягчение', 'снижаться', 'снижение', 'составлять',
       'сохранение', 'способствовать', 'спрос', 'ставка', 'существенно',
       'счет', 'также', 'темп', 'тенденция

In [49]:
bow[:, np.any(lr.coef_ != 0, axis=0)].shape

(96, 94)

In [50]:
X = np.concatenate((bow[:, np.any(lr.coef_ != 0, axis=0)].toarray(), df[['inflation', 'usd', 'usd_cur_change_relative']].values), axis=1)

calc_metrics(X, solver='newton-cg', penalty='l2')

(0.6363636363636364,
 LogisticRegression(C=1, max_iter=10000, solver='newton-cg'))

In [51]:
vectorizer_title = CountVectorizer(
    lowercase=True,
    stop_words=list(stopwords.words('russian')),
    tokenizer=word_tokenize,
    preprocessor=preprocessor,
)

vectorizer_title.fit(df.title)

bow_title = vectorizer_title.transform(df.title)

/home/father/Desktop/cbr-press-release-classifier/.venv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/father/Desktop/cbr-press-release-classifier/.venv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['весь', 'свой', 'это'] not in stop_words.
  warnings.warn(


In [52]:
metric, lr_title = calc_metrics(bow=bow_title, penalty='l2')
metric

0.6515151515151515

In [53]:
bow_title.toarray().shape

(96, 17)

In [54]:
vectorizer_title.get_feature_names_out()[np.any(lr_title.coef_ != 0, 0)]

array(['б', 'банк', 'годовой', 'ключевой', 'мера', 'операция', 'п',
       'повышать', 'принимать', 'процентный', 'пункт', 'решение',
       'россия', 'снижать', 'сохранять', 'ставка', 'уровень'],
      dtype=object)

In [57]:
X = np.concatenate((bow[:, np.any(lr.coef_ != 0, axis=0)].toarray(), bow_title.toarray()), axis=1)
X.shape

(96, 111)

In [59]:
metr, log_reg = calc_metrics(X, penalty='l2', solver='lbfgs',)
metr

0.6818181818181818

In [61]:
cur_pr

,title,release,inflation,rate,usd,usd_cur_change_relative,target_categorial,target_absolute,target_relative
date,,,,,,,,,
2024-10-25,Банк России принял решение повысить ключевую с...,Совет директоров Банка России 25 октября 2024 ...,8.63,NaN,96.7402,1.0618,NaN,NaN,NaN


In [60]:
log_reg.predict(cur_pr)

/home/father/Desktop/cbr-press-release-classifier/.venv/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


ValueError: could not convert string to float: 'Банк России принял решение повысить ключевую ставку на 200 б.п., до 21,00% годовых'

### Подготовка модели

Создадим пайплайн, который будет включать:
1. Векторизацию столбца текстов (`release`) с помощью алгоритма Bag of Words, все остальные столбцы отбрасываются;
2. Классификатор - модель логистической регрессии.

In [80]:
model = Pipeline([
    ('vectorizer',
        ColumnTransformer([
            ('bag_of_words', CountVectorizer(), 'release')
        ], remainder="drop")
    ),
    ('log_regression', LogisticRegression(max_iter=1000))
])

### Оценка качества модели

Поскольку датасет небольшой, используем кросс-валидацию для оценки качества модели. Разбиение на фолды будем делать со стратификацией, чтобы в тестовую выборку попадали объекты всех классов.

In [81]:
scoring = [
    'accuracy',
    'f1_weighted',
    'precision_weighted',
    'recall_weighted',
    'roc_auc_ovr',
    'roc_auc_ovo',
]

folds = StratifiedKFold(5, shuffle=True, random_state=RANDOM_STATE)
result = cross_validate(model, df, y, cv=folds, scoring=scoring, n_jobs=-1, return_train_score=True)

sc = pd.DataFrame(index=['mean', 'std'])
for score in scoring:
    scores = result['test_' + score]
    mean = scores.mean()
    s = pd.DataFrame({ score: [scores.mean(), scores.std()] }, index=['mean', 'std'])
    sc = pd.merge(sc, s, right_index=True, left_index=True)

sc

,accuracy,f1_weighted,precision_weighted,recall_weighted,roc_auc_ovr,roc_auc_ovo
mean,0.606316,0.599540,0.645545,0.606316,0.803627,0.817361
std,0.105809,0.110902,0.134030,0.105809,0.063028,0.057507


Средняя доля правильных ответов и F-мера среди всех тестовых данных приблизительно равны 0.6, стандартное отклонение $\approx 0.11$.

Выводы:
- Качество модели не слишком высоко
- Значение стандартного отклонения может говорить о небольшом переобучении.

### Предсказание следующей ключевой ставки

Теперь применим базовую модель к последнему пресс-релизу и определим прогноз по ключевой ставке на следующем заседании совета директоров ЦБ РФ 20 декабря 2024 г.:

In [82]:
model.fit(df, y)
model.predict_proba(cur_pr)

array([[0.00331283, 0.01892325, 0.97776392]])

Модель предсказывает решение о повышении ключевой ставки на следующем заседании с вероятностью 0.978.